# Netflix Show Cancellations: Reddit API Analysis TEST


## Step 1: Install Required Libraries + Set Up the Reddit API Connection
Before using the Reddit API, we need to install the `praw` library. It simplifies authentication and API interactions.

In [4]:
# Install the required library
!pip install praw

In [6]:
import praw
print("praw library imported successfully!")


praw library imported successfully!


In [8]:
!pip install python-dotenv
#This is needed to create an .env for your API credentials. We dont want random people having access to our keys!

In [9]:
import praw
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Retrieve credentials from environment variables
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
USER_AGENT = os.getenv('USER_AGENT')

# Authenticate with Reddit API
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)

# Test connection
print("Reddit API is read-only:", reddit.read_only)

Reddit API is read-only: True


## Step 2: Test Access to the Subreddits